In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
import matplotlib
import seaborn as sns
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Flatten, Softmax, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
train = pd.read_csv('train_final.csv')
test = pd.read_csv('test_28_final.csv')

In [ ]:
test_28 = pd.read_csv('test_28_final.csv')

In [ ]:
for col in ['metal_linker', 'organic_linker1', 'organic_linker2']:
    train[col] = train[col].astype('category')

## Explore Data

In [ ]:
check_col= ['volume [A^3]', 'weight [u]', 'density [g/cm^3]',
       'surface_area [m^2/g]', 'void_fraction', 'void_volume [cm^3/g]',
       'CO2/N2_selectivity',
       'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]']


In [ ]:
for col in check_col:
    min_,max_ = point_outlier(train[col])
    train.loc[train[col] < min_, col] = np.NAN
    train.loc[train[col] > max_, col] = np.NAN

In [ ]:
train = train.dropna()

In [ ]:
train.shape

(50909, 52)

## Train

In [ ]:
#train_x = train.drop(columns=['CO2_working_capacity [mL/g]','functional_groups','MOFname','smiles','MolWt','MolLogP','NumValenceElectrons','HeavyAtomCount','NOCount','NumAliphaticRings','NumAromaticRings','NumHAcceptors','NumHDonors','NumHeteroatoms','NumRotatableBonds'])
#train_y = train['CO2_working_capacity [mL/g]']

# train_x = train.drop(columns=['CO2_working_capacity [mL/g]','functional_groups','MOFname','smiles','MolecularFormula','InChI','InChIKey','IUPACName'])
# train_y = train['CO2_working_capacity [mL/g]']

train_x = train.drop(columns=['gsa','vsa','CO2_working_capacity [mL/g]','functional_groups','MOFname','smiles','MolecularFormula','CID','InChI','InChIKey','IUPACName'])
train_y = train['CO2_working_capacity [mL/g]']

In [ ]:
train_x = pd.get_dummies(train_x)

In [ ]:
train_x_column = train_x.columns

In [ ]:
train_x_column

Index(['volume [A^3]', 'weight [u]', 'density [g/cm^3]',
       'surface_area [m^2/g]', 'MolWt', 'MolLogP', 'NumValenceElectrons',
       'HeavyAtomCount', 'NOCount', 'NumAliphaticRings',
       ...
       'organic_linker2_59', 'topology_acs', 'topology_bcu', 'topology_etb',
       'topology_fof', 'topology_nbo', 'topology_pcu', 'topology_pts',
       'topology_sra', 'topology_the'],
      dtype='object', length=167)

### train set

In [ ]:
Xst, Xval, yst, yval = train_test_split(train_x,train_y, test_size =0.2)
sc = StandardScaler()
X=train_x
X =sc.fit_transform(X)
y=train_y #เราควรยึด distribution ของ traing set เป็นหลัก ถ้าใช้ distributionอื่นได้การฟิตด้วย ก้จะเกิดbiasได้
Xval = sc.transform (Xval)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight 

In [ ]:
!pip install xgboost

from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, RandomForestRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

Ada = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300),n_estimators = 200, random_state = 139)
Bagg = BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth=300), n_estimators = 200, random_state = 139)
Rforest = RandomForestRegressor(max_depth=20,random_state=139,n_estimators=300)
#Xgbr = xgb.XGBRegressor(subsample=0.7, n_estimators=1000, max_depth=10, learning_rate=0.01, colsample_bytree=0.4, colsample_bylevel=0.4)

level0 = list()
level0.append(('Ada', Ada))
level0.append(('Bagg', Bagg))
level0.append(('Rforest', Rforest))
#level0.append(('Xgbr', Xgbr))
# define meta learner model
level1 = LinearRegression()

model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
model.fit(X, y)

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
Best parameters: {'subsample': 0.7, 'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.5}
Lowest RMAE:  4.59013258284844


StackingRegressor(cv=5,
                  estimators=[('Ada',
                               AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=300),
                                                 n_estimators=200,
                                                 random_state=139)),
                              ('Bagg',
                               BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=300),
                                                n_estimators=200,
                                                random_state=139)),
                              ('Rforest',
                               RandomForestRegressor(max_depth=20,
                                                     n_estimators=300,
                                                     random_state=139)),
                              ('Xgbr'...
                                            interaction_constraints=None,
                                            learning_rate=

In [ ]:
from sklearn.metrics import mean_absolute_error
predict = clf.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.281450260774855

### val set

In [ ]:
for col in ['metal_linker', 'organic_linker1', 'organic_linker2']:
    pretest[col] = pretest[col].astype('category')
Xval_column = Xval.columns

l = []
for col in train_x_column:
    if col not in Xval_column:
        l.append(col)
listofzeros = [0] *2000
for col in l:
    Xval[col]= listofzeros
Xval = Xval.reindex(columns = train_x_column)
Xval = sc.transform (Xval)

NameError: name 'pretest' is not defined

### Test set

In [ ]:
for col in ['metal_linker', 'organic_linker1', 'organic_linker2']:
    test[col] = test[col].astype('category')
#test= test.drop(columns=['MOFname','functional_groups','smiles','MolecularFormula','InChI','InChIKey','IUPACName'])
test= test.drop(columns=['functional_groups','MOFname','smiles','MolecularFormula','CID','InChI','InChIKey','IUPACName'])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
test = pd.get_dummies(test)
test_column = test.columns

ValueError: Data must be 1-dimensional

In [ ]:
l = []
for col in train_x_column:
    if col not in test_column:
        l.append(col)
listofzeros = [0] *17000
for col in l:
    test[col]= listofzeros
test.shape

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
test = test.reindex(columns = train_x_column)

AttributeError: 'numpy.ndarray' object has no attribute 'reindex'

In [ ]:
test = sc.transform (test)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
pred = model.predict(test)

## Save to submission

In [ ]:
df = pd.DataFrame()
df['id'] = test_28['MOFname']
df['CO2_working_capacity [mL/g]'] = pred
df

,id,CO2_working_capacity [mL/g]
0,mof_unit_68614,-99.797081
1,mof_unit_68615,-82.516724
2,mof_unit_68616,6.269935
3,mof_unit_68617,18.221710
4,mof_unit_68618,-20.033360
...,...,...
16995,mof_unit_85609,-10.298024
16996,mof_unit_85610,-22.417246
16997,mof_unit_85611,-24.301266
16998,mof_unit_85612,-19.979942


In [ ]:
df['id'] = df['id'].apply(lambda x:x.replace("mof_unit_",""))

In [ ]:
df.to_csv('submission.csv',index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8e807ffc-5318-4bab-9b22-eec9246d8981' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>